In [31]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pydicom
import random 
from tqdm import tqdm

import os
import keras
import pydicom

from sklearn.preprocessing import LabelEncoder
import pickle

def read_pickle(filename):    
    with open(filename, 'rb') as fp:
        return pickle.load(fp)
    
def save_pickle(data,filename):       
    with open(filename, 'wb') as fp:
        pickle.dump(data, fp)  
    
data_path = '../input/rsna-pneumonia-detection-challenge/'

images_path = data_path + 'stage_2_train_images/'
labels_path = data_path + 'stage_2_train_labels.csv'

detailed_class_info_path = data_path + 'stage_2_detailed_class_info.csv'

class_encoder = LabelEncoder()
   
def merge_dataframes():
    df = pd.read_csv(labels_path)
    details_df = pd.read_csv(detailed_class_info_path)
    df = pd.concat([df,details_df.drop('patientId',1)], 1) 
    print(df.describe())
    print(df.shape[0], 'cases')

    return df


  
def load_ids_and_labels_from_file():
    ids = read_pickle('ids')
    labels = read_pickle('labels')
    return ids,labels
  
def get_ids_and_labels(num_class):
    df = merge_dataframes()
    df['class_id'] = class_encoder.fit_transform(df['class'])

    df.sort_values(by=['patientId', 'class_id'])
    
    ids = df.patientId.tolist()
    
    if(num_class==2):
        labels =  df.Target.tolist() 
    else:
        labels =  df.class_id.tolist() 
    
    save_pickle(ids, 'ids')
    save_pickle(labels, 'labels')

    return ids,labels

      

num_class = 3
ids,labels = get_ids_and_labels(num_class)



                 x            y      ...            height        Target
count  9555.000000  9555.000000      ...       9555.000000  30227.000000
mean    394.047724   366.839560      ...        329.269702      0.316108
std     204.574172   148.940488      ...        157.750755      0.464963
min       2.000000     2.000000      ...         45.000000      0.000000
25%     207.000000   249.000000      ...        203.000000      0.000000
50%     324.000000   365.000000      ...        298.000000      0.000000
75%     594.000000   478.500000      ...        438.000000      1.000000
max     835.000000   881.000000      ...        942.000000      1.000000

[8 rows x 5 columns]
30227 cases


In [32]:
from sklearn.model_selection import train_test_split

feature_tensors = np.load('../input/inception/inception_resnet_v2_features.npz')['arr_0']


y = labels[:feature_tensors.shape[0]]

X_train, X_test, y_train, y_test = train_test_split(feature_tensors, y, test_size=0.1, random_state=42)


In [33]:
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve
import matplotlib.pyplot as plt


def precision_recall(name, clf):  
    y_pred = clf.predict(X_test)
    
    if(num_class==2):
        roc_score = roc_auc_score(y_test, y_pred)
        print('roc_auc_score', roc_score)

    
    if(num_class==3):
        report = classification_report(y_test, y_pred, target_names=class_encoder.classes_)
    else:
        report = classification_report(y_test, y_pred)
        
    print('classification report for', name)
    print( report )
    

def evaluate_classifier(clf,name):    
    clf.fit(X_train,y_train)

    save_pickle(clf,name)

    precision_recall(name, clf)
    
    score = clf.score(X_test,y_test)
    
    print('average_score', round(score,3))

In [34]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
name = 'DecisionTreeClassifier'
evaluate_classifier(clf,name)     

classification report for DecisionTreeClassifier
                              precision    recall  f1-score   support

                Lung Opacity       0.63      0.76      0.69       987
No Lung Opacity / Not Normal       0.49      0.44      0.46      1136
                      Normal       0.41      0.37      0.39       898

                   micro avg       0.52      0.52      0.52      3021
                   macro avg       0.51      0.52      0.51      3021
                weighted avg       0.51      0.52      0.51      3021

average_score 0.523


In [35]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf = LinearDiscriminantAnalysis()

name = 'LinearDiscriminantAnalysis'

evaluate_classifier(clf,name) 

classification report for LinearDiscriminantAnalysis
                              precision    recall  f1-score   support

                Lung Opacity       0.56      0.53      0.55       987
No Lung Opacity / Not Normal       0.42      0.45      0.43      1136
                      Normal       0.48      0.45      0.47       898

                   micro avg       0.48      0.48      0.48      3021
                   macro avg       0.49      0.48      0.48      3021
                weighted avg       0.48      0.48      0.48      3021

average_score 0.479


In [36]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

clf = QuadraticDiscriminantAnalysis()

name = 'QuadraticDiscriminantAnalysis'

evaluate_classifier(clf,name) 

classification report for QuadraticDiscriminantAnalysis
                              precision    recall  f1-score   support

                Lung Opacity       0.65      0.72      0.68       987
No Lung Opacity / Not Normal       0.49      0.42      0.45      1136
                      Normal       0.49      0.52      0.50       898

                   micro avg       0.55      0.55      0.55      3021
                   macro avg       0.54      0.55      0.55      3021
                weighted avg       0.54      0.55      0.54      3021

average_score 0.548


In [37]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

name = 'RandomForestClassifier'

evaluate_classifier(clf,name) 

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


classification report for RandomForestClassifier
                              precision    recall  f1-score   support

                Lung Opacity       0.62      0.75      0.68       987
No Lung Opacity / Not Normal       0.47      0.49      0.48      1136
                      Normal       0.47      0.34      0.39       898

                   micro avg       0.53      0.53      0.53      3021
                   macro avg       0.52      0.53      0.52      3021
                weighted avg       0.52      0.53      0.52      3021

average_score 0.53


In [38]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

name = 'GaussianNB'

evaluate_classifier(clf,name) 

classification report for GaussianNB
                              precision    recall  f1-score   support

                Lung Opacity       0.45      0.56      0.50       987
No Lung Opacity / Not Normal       0.49      0.10      0.16      1136
                      Normal       0.38      0.67      0.49       898

                   micro avg       0.42      0.42      0.42      3021
                   macro avg       0.44      0.44      0.38      3021
                weighted avg       0.45      0.42      0.37      3021

average_score 0.418


In [39]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
name = 'LinearSVC'

evaluate_classifier(clf,name) 

classification report for LinearSVC
                              precision    recall  f1-score   support

                Lung Opacity       0.54      0.56      0.55       987
No Lung Opacity / Not Normal       0.41      0.42      0.41      1136
                      Normal       0.48      0.44      0.46       898

                   micro avg       0.47      0.47      0.47      3021
                   macro avg       0.48      0.47      0.47      3021
                weighted avg       0.47      0.47      0.47      3021

average_score 0.472


/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [47]:

from sklearn.model_selection import train_test_split

y = labels[:feature_tensors.shape[0]]
y = np.array(y)
y = keras.utils.to_categorical(y, num_class)

X_train, X_test, y_train, y_test = train_test_split(feature_tensors, y, test_size=0.2, random_state=42)

from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten,BatchNormalization,LeakyReLU
from keras.metrics import categorical_accuracy, binary_accuracy

if(num_class==2):
    print(2)
    model = Sequential()
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[binary_accuracy])

else:
    print(3)

    model = Sequential()
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=[categorical_accuracy])



model.fit(X_train,y_train,
          epochs=12,
          batch_size=16,
          validation_data=(X_test, y_test)
         )

model.save('inception.h5') 

3
Train on 24166 samples, validate on 6042 samples
Epoch 1/12
24166/24166 [==============================] - 7s 294us/step - loss: 1.0717 - categorical_accuracy: 0.4069 - val_loss: 1.0408 - val_categorical_accuracy: 0.4346
Epoch 2/12
24166/24166 [==============================] - 6s 238us/step - loss: 1.0448 - categorical_accuracy: 0.4304 - val_loss: 1.0295 - val_categorical_accuracy: 0.4416
Epoch 3/12
24166/24166 [==============================] - 6s 230us/step - loss: 1.0410 - categorical_accuracy: 0.4325 - val_loss: 1.0228 - val_categorical_accuracy: 0.4424
Epoch 4/12
24166/24166 [==============================] - 6s 233us/step - loss: 1.0375 - categorical_accuracy: 0.4337 - val_loss: 1.0346 - val_categorical_accuracy: 0.4500
Epoch 5/12
24166/24166 [==============================] - 6s 229us/step - loss: 1.0362 - categorical_accuracy: 0.4363 - val_loss: 1.0351 - val_categorical_accuracy: 0.4568
Epoch 6/12
24166/24166 [==============================] - 6s 239us/step - loss: 1.0347 - 